# Etude de l'influence de paramètres sur l'efficacité d'attaques

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join


import numpy as np
import pandas as pd
import cv2
import pickle
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf



from tensorflow import convert_to_tensor
import tensorflow.keras.backend as K
from tensorflow.keras.models import  load_model,Model
from tensorflow.keras import applications
from tensorflow.keras.layers import Dropout, Activation, Dense, GlobalMaxPooling2D,Conv2D,Flatten,MaxPooling2D,InputLayer
from tensorflow.keras.utils import to_categorical

from efficientnet.tfkeras import EfficientNetB7
from tensorflow.keras import datasets
from foolbox.attacks import LinfFastGradientAttack,LinfDeepFoolAttack
from foolbox.distances import LpDistance
from foolbox.models import TensorFlowModel
from foolbox import criteria
from sklearn.metrics import classification_report,plot_roc_curve

from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import BoundaryAttack
from art.utils import load_dataset #to play with cifar images

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6*1024)]
)


3 facteurs pouvant influencer la perfomance d'attaques adverses ont été identifié:
- **La complexité du modèle (small custom model vs efficientnet)**
- **La complexité de l'attaque**
- **La complexité et la variance interne du jeu de données**

Nous avons choisi d'étudier différent scénario pour lequel nous allons tracer à chaque fois le success rate de l'attaque en fonction du degree of change. Nous étudierons:
- **3 attaques (deepfool 1 step, deepfool mutliple steps et FGSM)**
- **2 modèles (efficientNet et small custom model)**
- **2 jeux de données (Mnist et Cifar)**

### 1. Load Data

#### Mnist

In [3]:
data_mnist=datasets.mnist.load_data(path='mnist.npz')
X_train_mnist,y_train_mnist=data_mnist[0][0],data_mnist[0][1]
X_test_mnist,y_test_mnist=data_mnist[1][0],data_mnist[1][1]
y_train_mnist = to_categorical(y_train_mnist, 10)
y_test_mnist = to_categorical(y_test_mnist, 10)

""" Train """
X_train_img_mnist = np.full((60000, 32, 32, 3), 0)
for i, s in enumerate(X_train_mnist):
    X_train_img_mnist[i] = cv2.cvtColor(np.pad(s,2), cv2.COLOR_GRAY2RGB) 
    
""" Test """
X_test_img_mnist = np.full((10000, 32, 32, 3), 0)
for i, s in enumerate(X_test_mnist):
    X_test_img_mnist[i] = cv2.cvtColor(np.pad(s,2), cv2.COLOR_GRAY2RGB) 
    
X_train_mnist=X_train_img_mnist
X_test_mnist= X_test_img_mnist

#### Cifar-10

In [4]:
(X_train_cifar, y_train_cifar), (X_test_cifar, y_test_cifar), _,_=load_dataset('cifar10')
for i, im in enumerate(X_train_cifar):
    X_train_cifar[i]=255*im
for i, im in enumerate(X_test_cifar):
    X_test_cifar[i]=255*im    


dict_classes={}
dict_classes[0]='airplane'
dict_classes[1]='automobile'
dict_classes[2]='bird'
dict_classes[3]='cat'
dict_classes[4]='deer'
dict_classes[5]='dog'
dict_classes[6]='frog'
dict_classes[7]='horse'
dict_classes[8]='ship'
dict_classes[9]='truck'

#### 2. Load models

In [5]:
def pick_data_set(name):
    
    if name=='Mnist':
        X_train=X_train_mnist
        X_test= X_test_mnist
        y_train = y_train_mnist
        y_test= y_test_mnist

    elif name=='Cifar':
        X_train=X_train_cifar
        X_test= X_test_cifar
        y_train = y_train_cifar
        y_test= y_test_cifar    
    return(X_train,X_test,y_train,y_test)
    
    

Nous allons comparer les attaques sur efficientNet,sur un petit modèle 'fait maison'  

In [6]:
def train_and_save_effnet(data_set_name,training_effnet=True):
    (X_train,X_test,y_train,y_test)=pick_data_set(data_set_name)
    tf.keras.backend.clear_session()
    effnet_base = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    effnet_base.trainable=True
    x = GlobalMaxPooling2D(name='pool_1')(effnet_base.layers[-2].output)
    x = Dropout(0.2, name="dropout_2")(x)
    x = Dense(32)(x)
    x = Dense(10,name='fc_2')(x)
    o = Activation('softmax', name='act_2')(x)
    model_effnet = Model(inputs=effnet_base.input, outputs=[o])


    training_effnet=training_effnet

    if training_effnet==True:
        model_effnet.compile(
            loss='categorical_crossentropy',
            optimizer='nadam',
            metrics=['accuracy']
            )
        history = model_effnet.fit(X_train, y_train,
                      epochs=5,
                      batch_size = 128,
                      validation_split=0.1,
                      shuffle=True,
                      verbose=1)
        model_effnet.save('/media/hdd1/benchmark_adversarial_attacks/effnet_model_'+str(name)+'.h5')

    else:
        model_effnet=load_model('/media/hdd1/benchmark_adversarial_attacks/effnet_model_'+str(name)+'.h5')
        
    return(model_effnet)

    

In [7]:
def train_and_save_small_model(data_set_name):
    
    (X_train,X_test,y_train,y_test)=pick_data_set(data_set_name)
    tf.keras.backend.clear_session()   
    small_model = tf.keras.models.Sequential()
    small_model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32,32,3)))
    small_model.add(MaxPooling2D(2, 2))
    small_model.add(Conv2D(64, (3, 3), activation='relu'))
    small_model.add(MaxPooling2D(2, 2))
    small_model.add(Flatten())
    small_model.add(Dense(128, activation='relu'))
    small_model.add(Dense(10, activation='softmax'))


    small_model.compile(
        loss='categorical_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
        )


    history = small_model.fit(X_train, y_train,
                  epochs=10,
                  batch_size =128,
                  validation_split=0.1,
                  shuffle=True,
                  verbose=1)


    small_model.save('/media/hdd1/benchmark_adversarial_attacks/small_model_'+str(name)+'.h5')
    return(small_model)

#### 3. Load attacks 

contrast_enhancer permet d augmenter le bruit lorsqu il est très faible

In [8]:
def contrast_enhancer(image_noise):    
    for i in range (3):
        color_axis=image_noise[:,:,i]
        color_axis =((color_axis -np.min(color_axis))/np.ptp(color_axis))
        image_noise[:,:,i]=color_axis
    image_noise=image_noise #in order to have the darker colors corresponding to the biggest changes
    return(image_noise)

Let's define the peformances metrics

In [9]:
def DOC (x_adv_list,x_list): #degree of change
    N=len(x_adv_list)
    sum=0
    for i in range (N):        
        sum+=100*np.linalg.norm(np.reshape(x_adv_list[i] - x_list[i], -1),ord=1)/np.linalg.norm(np.reshape(x_list[i], -1),ord=1)
    return(sum/N)

def succes_rate (success_list):
    success=len([i for i in success_list if i==True])
    total=len(success_list)
    return(success/total)

In [10]:
attack_deepfool_1_steps=LinfDeepFoolAttack(steps=1)
attack_deepfool_mutliple_steps=LinfDeepFoolAttack()
attack_FGSM=LinfFastGradientAttack()

#### 5. Attacks performances

In [11]:
def black_box_attack(model_to_attack,target,iterations=150):
    classifier = TensorFlowV2Classifier(model=model_to_attack, input_shape=(32,32,3),clip_values=(0, 255),nb_classes=10)
    SR_blackbox_dic={}
    attack = BoundaryAttack(estimator=classifier, targeted=False, max_iter=0, delta=0.001, epsilon=0.001,init_size=100)
    iter_step =1
    x_adv = None
    for i in range(iterations):
        x_adv = attack.generate(x=np.array([target]), x_adv_init=x_adv)

        #clear_output()
        print("Adversarial image at step %d." % (i * iter_step), "L2 error", 
              np.linalg.norm(np.reshape(x_adv[0] - target, [-1])),
              "and class label %d." % np.argmax(classifier.predict(x_adv)[0]))
        plt.imshow(x_adv[0][..., ::-1].astype('int32'))
        plt.show(block=False)
        SR_blackbox_dic[DOC([x_adv[0]],[target])]=1

        if hasattr(attack, 'curr_delta') and hasattr(attack, 'curr_epsilon'):
            attack.max_iter = iter_step 
            attack.delta = attack.curr_delta
            attack.epsilon = attack.curr_epsilon
        else:
            break
    return(SR_blackbox_dic)


attack_performances_computer renvoit le degree of change d'une attaque et son success rate

In [12]:
def attack_performances_computer(model_to_fool,attack, image_list, labels,epsilon):
    if len(labels[0])>1:
        labels=list(map(np.argmax,labels))
    success=[]
    adv_list=[]
    attack_value=[]
    for i,image in enumerate(tqdm(image_list,position=0)):
        image = np.asarray(image)[:,:,:3].astype('float32')
        image = convert_to_tensor(np.expand_dims(image,axis=0))
        label=labels[i]
        label = tf.convert_to_tensor(np.array([label]))
        _, clipped, is_adv = attack(model_to_fool,image,label,epsilons=epsilon)
        success.append(bool(is_adv[0][0]))
        adv_list.append(np.array(clipped[0][0]))
    DOC_attack=DOC(adv_list,image_list)
    SR_attack=succes_rate(success)
    return(DOC_attack,SR_attack)

##### Comparison FGSM vs deepfool 1 step vs deepfool mutliple steps on effnet

The success rate with 5 steps is the same than with one step, however the DOC is slightly better (lower) with one step. 

In [13]:
def attack_runner(model_to_fool,image_list, labels_list, epislons_list):
    
    
    attack_deepfool_1_steps=LinfDeepFoolAttack(steps=1)
    attack_deepfool_mutliple_steps=LinfDeepFoolAttack()
    attack_FGSM=LinfFastGradientAttack()
    
    if len(labels_list[0])<=5:
        labels_list=list(map(np.argmax,labels_list))

    SR_FGSM_dic={}
    SR_deepfool_1step_dic={}
    SR_deepfool_mutliple_steps_dic={}


    for epsilon in epislons_list:
        print('======'+'epislon: '+str(epsilon)+'======')
        DOC_FGSM,SR_FGSM=attack_performances_computer(model_to_fool,attack_FGSM, image_list, labels_list,[epsilon])
        DOC_deepfool_1step,SR_deepfool_1step=attack_performances_computer(model_to_fool,attack_deepfool_1_steps, image_list, labels_list,[epsilon])
        DOC_deepfool_mutliple_steps,SR_deepfool_mutliple_steps=attack_performances_computer(model_to_fool,attack_deepfool_mutliple_steps, image_list, labels_list,[epsilon])


        SR_deepfool_1step_dic[DOC_deepfool_1step]=SR_deepfool_1step   
        SR_deepfool_mutliple_steps_dic[DOC_deepfool_mutliple_steps]=SR_deepfool_mutliple_steps    
        SR_FGSM_dic[DOC_FGSM]=SR_FGSM
        
    return(SR_deepfool_1step_dic,SR_deepfool_mutliple_steps_dic,SR_FGSM_dic)

In [14]:
epsilon_list=[0.001,0.01,0.1,1,10,25,50,100]



name='Mnist'
(X_train,X_test,y_train,y_test)=pick_data_set(name)
small_model=train_and_save_small_model(name)

model_to_fool = TensorFlowModel(small_model, bounds=(0, 255))
(SR_deepfool_1step_dic_small_model,SR_deepfool_mutliple_steps_dic_small_model,SR_FGSM_dic_small_model)=attack_runner(model_to_fool,X_test[:100], y_test[:100], epsilon_list)
SR_blackbox_dic_small_model=black_box_attack(small_model,X_test[0].astype('int32'))
with open('data/perfs_global_small_model_'+str(name), 'wb') as f:
    pickle.Pickler(f).dump(SR_deepfool_1step_dic_small_model)
    pickle.Pickler(f).dump(SR_deepfool_mutliple_steps_dic_small_model)
    pickle.Pickler(f).dump(SR_FGSM_dic_small_model)
    pickle.Pickler(f).dump(SR_blackbox_dic_small_model)

Epoch 1/10
422/422 [==============================] - 3s 8ms/step - loss: 0.9911 - accuracy: 0.9230 - val_loss: 0.0745 - val_accuracy: 0.9798
Epoch 2/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0687 - accuracy: 0.9800 - val_loss: 0.0680 - val_accuracy: 0.9828
Epoch 3/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0434 - accuracy: 0.9864 - val_loss: 0.0782 - val_accuracy: 0.9797
Epoch 4/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0327 - accuracy: 0.9898 - val_loss: 0.0647 - val_accuracy: 0.9847
Epoch 5/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0295 - accuracy: 0.9911 - val_loss: 0.0622 - val_accuracy: 0.9862
Epoch 6/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0258 - accuracy: 0.9917 - val_loss: 0.0707 - val_accuracy: 0.9858
Epoch 7/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0235 - accuracy: 0.9927 - val_loss: 0.0750 - val_accuracy: 0.9825
Epoch 

 10%|█         | 10/100 [00:00<00:00, 96.09it/s]

======epislon: 0.001======


 16%|█▌        | 16/100 [00:00<00:00, 153.86it/s]

======epislon: 0.01======


...

======epislon: 50======


 16%|█▌        | 16/100 [00:00<00:00, 154.30it/s]

======epislon: 100======


 84%|████████▍ | 84/100 [00:12<00:02,  5.58it/s]

KeyboardInterrupt: 

In [ ]:
name='Mnist'
(X_train,X_test,y_train,y_test)=pick_data_set(name)
y_pred_small_model=[np.argmax(pred) for pred in small_model.predict(X_test)]
y_true=[np.argmax(y) for y in y_test]
classification_small_model=classification_report(y_true, y_pred_small_model)
print('performances of the small model on '+str(name))
print('-------------------------')
print(classification_small_model)

In [ ]:
epsilon_list=[0.001,0.01,0.1,1,10,25,50,100]


name='Mnist'
(X_train,X_test,y_train,y_test)=pick_data_set(name)

model_effnet=train_and_save_effnet(name)
model_to_fool = TensorFlowModel(model_effnet,bounds=(0, 255))

(SR_deepfool_1step_dic_model_effnet,SR_deepfool_mutliple_steps_dic_model_effnet,SR_FGSM_dic_model_effnet)=attack_runner(model_to_fool,X_test[:100], y_test[:100], epsilon_list)
SR_blackbox_dic_model_effnet=black_box_attack(model_effnet,X_test[0],iterations=150)
with open('data/perfs_global_model_effnet_'+str(name), 'wb') as f:
    pickle.Pickler(f).dump(SR_deepfool_1step_dic_model_effnet)
    pickle.Pickler(f).dump(SR_deepfool_mutliple_steps_dic_model_effnet)
    pickle.Pickler(f).dump(SR_FGSM_dic_model_effnet)
    pickle.Pickler(f).dump(SR_blackbox_dic_model_effnet)

In [ ]:
y_pred_eff=[np.argmax(pred) for pred in model_effnet.predict(X_test)]
y_true=[np.argmax(y) for y in y_test]
classification_eff=classification_report(y_true, y_pred_eff)
print('performances of effnet on '+str(name))
print('-------------------------')
print(classification_eff)

In [ ]:
tf.compat.v1.reset_default_graph()

In [ ]:
epsilon_list=[0.001,0.01,0.1,1,10,25,50,100]



name='Cifar'
(X_train,X_test,y_train,y_test)=pick_data_set(name)
small_model=train_and_save_small_model(name)

model_to_fool = TensorFlowModel(small_model, bounds=(0, 255))
(SR_deepfool_1step_dic_small_model,SR_deepfool_mutliple_steps_dic_small_model,SR_FGSM_dic_small_model)=attack_runner(model_to_fool,X_test[:100], y_test[:100], epsilon_list)
SR_blackbox_dic_small_model=black_box_attack(small_model,X_test[0])
with open('data/perfs_global_small_model_'+str(name), 'wb') as f:
    pickle.Pickler(f).dump(SR_deepfool_1step_dic_small_model)
    pickle.Pickler(f).dump(SR_deepfool_mutliple_steps_dic_small_model)
    pickle.Pickler(f).dump(SR_FGSM_dic_small_model)
    pickle.Pickler(f).dump(SR_blackbox_dic_small_model)

In [ ]:
name='Cifar'
(X_train,X_test,y_train,y_test)=pick_data_set(name)
y_pred_small_model=[np.argmax(pred) for pred in small_model.predict(X_test)]
y_true=[np.argmax(y) for y in y_test]
classification_small_model=classification_report(y_true, y_pred_small_model)
print('performances of the small model on '+str(name))
print('-------------------------')
print(classification_small_model)

In [ ]:
epsilon_list=[0.001,0.01,0.1,1,10,25,50,100]

name='Cifar'
(X_train,X_test,y_train,y_test)=pick_data_set(name)

model_effnet=train_and_save_effnet(name)
model_to_fool = TensorFlowModel(model_effnet, bounds=(0, 255))

(SR_deepfool_1step_dic_model_effnet,SR_deepfool_mutliple_steps_dic_model_effnet,SR_FGSM_dic_model_effnet)=attack_runner(model_to_fool,X_test[:100], y_test[:100], epsilon_list)
SR_blackbox_dic_model_effnet=black_box_attack(model_effnet,X_test[0],iterations=150)
with open('data/perfs_global_model_effnet_'+str(name), 'wb') as f:
    pickle.Pickler(f).dump(SR_deepfool_1step_dic_model_effnet)
    pickle.Pickler(f).dump(SR_deepfool_mutliple_steps_dic_model_effnet)
    pickle.Pickler(f).dump(SR_FGSM_dic_model_effnet)
    pickle.Pickler(f).dump(SR_blackbox_dic_model_effnet)

In [ ]:
y_pred_eff=[np.argmax(pred) for pred in model_effnet.predict(X_test)]
y_true=[np.argmax(y) for y in y_test]
classification_eff=classification_report(y_true, y_pred_eff)
print('performances of effnet on '+str(name))
print('-------------------------')
print(classification_eff)

In [ ]:
with open('data/perfs_global_small_model_Cifar', 'rb') as f:
    SR_deepfool_1step_dic_small_model_Cifar=pickle.Unpickler(f).load()
    SR_deepfool_mutliple_steps_dic_small_model_Cifar=pickle.Unpickler(f).load()
    SR_FGSM_dic_small_model_Cifar=pickle.Unpickler(f).load()
    SR_blackbox_dic_small_model_Cifar=pickle.Unpickler(f).load()
    
with open('data/perfs_global_model_effnet_Cifar', 'rb') as f:
    SR_deepfool_1step_dic_model_effnet_Cifar=pickle.Unpickler(f).load()
    SR_deepfool_mutliple_steps_dic_model_effnet_Cifar=pickle.Unpickler(f).load()
    SR_FGSM_dic_model_effnet_Cifar=pickle.Unpickler(f).load()
    SR_blackbox_dic_model_effnet_Cifar=pickle.Unpickler(f).load()
    
with open('data/perfs_global_small_model_Mnist', 'rb') as f:
    SR_deepfool_1step_dic_small_model_Mnist=pickle.Unpickler(f).load()
    SR_deepfool_mutliple_steps_dic_small_model_Mnist=pickle.Unpickler(f).load()
    SR_FGSM_dic_small_model_Mnist=pickle.Unpickler(f).load()
    SR_blackbox_dic_small_model_Mnist=pickle.Unpickler(f).load()
    

with open('data/perfs_global_model_effnet_Mnist', 'rb') as f:
    SR_deepfool_1step_dic_model_effnet_Mnist=pickle.Unpickler(f).load()
    SR_deepfool_mutliple_steps_dic_model_effnet_Mnist=pickle.Unpickler(f).load()
    SR_FGSM_dic_model_effnet_Mnist=pickle.Unpickler(f).load()
    SR_blackbox_dic_model_effnet_Mnist=pickle.Unpickler(f).load()
    
   

In [ ]:
plt.rcParams["figure.figsize"] = 21,14
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
fig, ax = plt.subplots(ncols=3, nrows=3, gridspec_kw={"height_ratios":[0.01,1,1],"width_ratios":[0.5,1,1]})
fig.suptitle('Parameters influence on attack effectiveness', 
             fontsize=20, fontweight='bold')


ax[0][0].axis("off")
ax[0][0].set_title("Data Set / Model", fontweight='bold',fontsize=15)


ax[0][1].axis("off")
ax[0][1].set_title("Small Model", fontweight='bold',fontsize=15)


ax[0][2].axis("off")
ax[0][2].set_title("EfficentNet", fontweight='bold',fontsize=15)


ax[1][0].axis("off")
ax[1][0].set_title("Cifar Data set", fontweight='bold',fontsize=15)



ax[2][0].axis("off")
ax[2][0].set_title("Mnist Data set", fontweight='bold',fontsize=15)


ax[1][1].plot(list(SR_deepfool_1step_dic_small_model_Cifar.keys()),list(SR_deepfool_1step_dic_small_model_Cifar.values()),label='deepfool 1 step')
ax[1][1].plot(list(SR_deepfool_mutliple_steps_dic_small_model_Cifar.keys()),list(SR_deepfool_mutliple_steps_dic_small_model_Cifar.values()),label='Deepfool multiple steps')
ax[1][1].plot(list(SR_FGSM_dic_small_model_Cifar.keys()),list(SR_FGSM_dic_small_model_Cifar.values()),label='FGSM')
ax[1][1].plot(list(SR_blackbox_dic_small_model_Cifar.keys()),list(SR_blackbox_dic_small_model_Cifar.values()),label='boundary_attack')
ax[1][1].set_xscale('log')
ax[1][1].grid(True,which="both", linestyle='--')
ax[1][1].set_title('Successs rate for different degrees of change \n on Cifar images for the small model',fontsize=11)
ax[1][1].set_xlabel('DOC (%)')
ax[1][1].set_ylabel('SR')
ax[1][1].legend(loc='lower right')


ax[1][2].plot(list(SR_deepfool_1step_dic_model_effnet_Cifar.keys()),list(SR_deepfool_1step_dic_model_effnet_Cifar.values()),label='deepfool 1 step')
ax[1][2].plot(list(SR_deepfool_mutliple_steps_dic_model_effnet_Cifar.keys()),list(SR_deepfool_mutliple_steps_dic_model_effnet_Cifar.values()),label='Deepfool multiple steps')
ax[1][2].plot(list(SR_FGSM_dic_model_effnet_Cifar.keys()),list(SR_FGSM_dic_model_effnet_Cifar.values()),label='FGSM')
ax[1][2].plot(list(SR_blackbox_dic_model_effnet_Cifar.keys()),list(SR_blackbox_dic_model_effnet_Cifar.values()),label='boundary_attack')
ax[1][2].set_xscale('log')
ax[1][2].grid(True,which="both", linestyle='--')
ax[1][2].set_title('Successs rate for different degrees of change \n on Cifar images for the effnet model',fontsize=11)
ax[1][2].set_xlabel('DOC (%)')
ax[1][2].set_ylabel('SR')
ax[1][2].legend(loc='lower right')






ax[2][1].plot(list(SR_deepfool_1step_dic_small_model_Mnist.keys()),list(SR_deepfool_1step_dic_small_model_Mnist.values()),label='deepfool 1 step')
ax[2][1].plot(list(SR_deepfool_mutliple_steps_dic_small_model_Mnist.keys()),list(SR_deepfool_mutliple_steps_dic_small_model_Mnist.values()),label='Deepfool multiple steps')
ax[2][1].plot(list(SR_FGSM_dic_small_model_Mnist.keys()),list(SR_FGSM_dic_small_model_Mnist.values()),label='FGSM')
ax[2][1].plot(list(SR_blackbox_dic_small_model_Mnist.keys()),list(SR_blackbox_dic_small_model_Mnist.values()),label='boundary_attack')
ax[2][1].set_xscale('log')
ax[2][1].grid(True,which="both", linestyle='--')
ax[2][1].set_title('Successs rate for different degrees of change \n on Mnist images for the small model',fontsize=11)
ax[2][1].set_xlabel('DOC (%)')
ax[2][1].set_ylabel('SR')
ax[2][1].legend(loc='upper left')


ax[2][2].plot(list(SR_deepfool_1step_dic_model_effnet_Mnist.keys()),list(SR_deepfool_1step_dic_model_effnet_Mnist.values()),label='deepfool 1 step')
ax[2][2].plot(list(SR_deepfool_mutliple_steps_dic_model_effnet_Mnist.keys()),list(SR_deepfool_mutliple_steps_dic_model_effnet_Mnist.values()),label='Deepfool multiple steps')
ax[2][2].plot(list(SR_FGSM_dic_model_effnet_Mnist.keys()),list(SR_FGSM_dic_model_effnet_Mnist.values()),label='FGSM')
ax[2][2].plot(list(SR_blackbox_dic_model_effnet_Mnist.keys()),list(SR_blackbox_dic_model_effnet_Mnist.values()),label='boundary_attack')
ax[2][2].set_xscale('log')
ax[2][2].grid(True,which="both", linestyle='--')
ax[2][2].set_title('Successs rate for different degrees of change \n on Mnist images for the effnet model',fontsize=11)
ax[2][2].set_xlabel('DOC (%)')
ax[2][2].set_ylabel('SR')
ax[2][2].legend(loc='upper left')





fig.subplots_adjust(hspace=0.5,bottom=0.1)
plt.show()